# Optimización de la Cadena de Suministro de Banano en Ecuador

## Contexto
Ecuador, reconocido como líder mundial en la exportación de banano, enfrenta el desafío de optimizar su cadena de suministro. El país necesita gestionar de manera eficiente el transporte de banano desde las principales zonas productoras en las provincias de El Oro, Guayas y Los Ríos hasta sus puertos de exportación. Este ejercicio propone un modelo complejo que considera varias plantaciones y la demanda en los puertos, influenciada por las rutas comerciales internacionales.

## Objetivo
El principal objetivo es minimizar los costos totales de transporte. Esto incluye considerar las capacidades de producción en cada plantación, las necesidades de los mercados internacionales y la disponibilidad de rutas de transporte marítimo.

## Datos del Problema

### Capacidad de Producción de las Plantaciones (en toneladas)
- **El Oro**:
  - Machala: 700 ton
  - Pasaje: 650 ton
- **Guayas**:
  - Naranjal: 750 ton
  - Milagro: 775 ton
- **Los Ríos**:
  - Babahoyo: 650 ton
  - Quevedo: 725 ton

### Demanda en los Puertos (desglosada por país y estado de EE.UU., en toneladas)
- **Guayaquil**:
  - España: 350 ton
  - Alemania: 250 ton
  - Florida, EE.UU.: 500 ton
- **Manta**:
  - México: 450 ton
  - Italia: 200 ton
  - California, EE.UU.: 300 ton
- **Puerto Bolívar**:
  - Estados Unidos (Texas): 400 ton
  - Canadá: 300 ton
  - Reino Unido: 300 ton
- **Esmeraldas**:
  - Francia: 250 ton
  - Reino Unido: 200 ton
  - Japón: 350 ton

### Costos de Transporte (USD por tonelada)
Los costos de transporte se calcularán basándose en la distancia desde cada plantación hasta los distintos puertos, asumiendo un costo promedio de $0.20 USD por kilómetro por tonelada.

## Tareas
- Modelar el problema utilizando la programación lineal.
- Definir variables de decisión para las toneladas de banano a transportar desde cada plantación a cada puerto.
- Establecer una función objetivo para minimizar el costo total del transporte.
- Incluir restricciones para respetar la capacidad de producción y satisfacer la demanda de cada puerto.
- Analizar los resultados para entender la distribución óptima y el costo total del transporte.

Este ejercicio es una oportunidad para aplicar conocimientos avanzados de programación lineal y logística en un contexto real y complejo.


In [105]:
# Importar bibliotecas

In [106]:
from pyomo.environ import *

In [107]:
# Datos del problema

In [108]:
# Oferta
# Capacidad en Toneladas de las diferenets fincas

In [109]:
Oferta = {
    'Machala': 700, 'Pasaje': 650,
    'Naranjal': 750, 'Milagro': 775,
    'Babahoyo': 650, 'Quevedo': 725
}

In [110]:
# Demanda Agregada por Puerto.
# Recordemos, la demanda no la genera el puerto sino el país con una "conexión" con un puerto determinado.
# Lo que presentamos es una demanda agregada por puerto.

In [111]:
Demanda = {
    'Guayaquil': 1100,
    'Manta': 950,
    'Puerto Bolívar': 1000,
    'Esmeraldas': 800 
}

In [112]:
# Costos

In [113]:
Costos = {
 ('Machala', 'Guayaquil'): 38.0,
 ('Machala', 'Manta'): 58.0,
 ('Machala', 'Puerto Bolívar'): 24.0,
 ('Machala', 'Esmeraldas'): 84.0,
 ('Pasaje', 'Guayaquil'): 36.0,
 ('Pasaje', 'Manta'): 56.0,
 ('Pasaje', 'Puerto Bolívar'): 22.0,
 ('Pasaje', 'Esmeraldas'): 82.0,
 ('Naranjal', 'Guayaquil'): 12.0,
 ('Naranjal', 'Manta'): 32.0,
 ('Naranjal', 'Puerto Bolívar'): 30.0,
 ('Naranjal', 'Esmeraldas'): 60.0,
 ('Milagro', 'Guayaquil'): 8.0,
 ('Milagro', 'Manta'): 28.0,
 ('Milagro', 'Puerto Bolívar'): 26.0,
 ('Milagro', 'Esmeraldas'): 56.0,
 ('Babahoyo', 'Guayaquil'): 14.0,
 ('Babahoyo', 'Manta'): 34.0,
 ('Babahoyo', 'Puerto Bolívar'): 32.0,
 ('Babahoyo', 'Esmeraldas'): 62.0,
 ('Quevedo', 'Guayaquil'): 34.0,
 ('Quevedo', 'Manta'): 54.0,
 ('Quevedo', 'Puerto Bolívar'): 52.0,
 ('Quevedo', 'Esmeraldas'): 72.0
}

In [114]:
# Listas de fincas y puertos

In [115]:
FINCAS = list(Oferta.keys())

In [116]:
PUERTOS = list(Demanda.keys())

In [117]:
# Crear el modelo de Pyomo

In [118]:
modelo = ConcreteModel()

In [119]:
# Variables de decisión: toneladas de banano a transportar desde cada plantación a cada puerto

In [120]:
modelo.x = Var(FINCAS, PUERTOS, domain=NonNegativeReals)

In [121]:
# Función objetivo: Minimizar costo de transporte

In [122]:
modelo.CostoTransporte = Objective(
    expr=sum(Costos[finca,puerto]*modelo.x[finca,puerto]
            for finca in FINCAS
            for puerto in PUERTOS),
    sense=minimize)

In [123]:
# Restricciones de capacidad de producción de las plantaciones

In [124]:
modelo.restriccionProduccion = ConstraintList()
for finca in FINCAS:
    modelo.restriccionProduccion.add(
        sum(modelo.x[finca, puerto] for puerto in PUERTOS) <= Oferta[finca])

In [125]:
# Restricciones para satisfacer la demanda de cada puerto

In [126]:
modelo.restriccionDemanda = ConstraintList()
for puerto in PUERTOS:
    modelo.restriccionDemanda.add(
        sum(modelo.x[finca,puerto] for finca in FINCAS) == Demanda[puerto])

In [127]:
# Resolver el modelo

In [128]:
solucion = SolverFactory('glpk').solve(modelo)

In [129]:
# Imprimir resultados

In [130]:
print("Costo Total de Transporte es USD:", value(modelo.CostoTransporte))
for finca in FINCAS:
    for puerto in PUERTOS:
        if value(modelo.x[finca,puerto]) > 0:
            print("Envíe desde la finca en", finca, "al puerto", puerto, value(modelo.x[finca,puerto]), "Toneladas")


Costo Total de Transporte es USD: 120600.0
Envíe desde la finca en Machala al puerto Puerto Bolívar 350.0 Toneladas
Envíe desde la finca en Pasaje al puerto Puerto Bolívar 650.0 Toneladas
Envíe desde la finca en Naranjal al puerto Guayaquil 750.0 Toneladas
Envíe desde la finca en Milagro al puerto Manta 650.0 Toneladas
Envíe desde la finca en Milagro al puerto Esmeraldas 125.0 Toneladas
Envíe desde la finca en Babahoyo al puerto Guayaquil 350.0 Toneladas
Envíe desde la finca en Babahoyo al puerto Manta 300.0 Toneladas
Envíe desde la finca en Quevedo al puerto Esmeraldas 675.0 Toneladas
